# Capstone 1 (from Chapter 5) — Monte Carlo Estimation of Pi

## Setup

In [1]:
import logging
import os

from llm_agents_from_scratch.logger import enable_console_logging

### Constants

In [2]:
IS_ON_RUNPOD = "RUNPOD_POD_ID" in os.environ
LOGGING_ENABLED = True
LOGGING_LEVEL = logging.INFO

# for task execution
MAX_STEPS = 20
NUM_REPLICATIONS = 10

In [3]:
# Install additional dependencies for notebook
if IS_ON_RUNPOD:
    !uv pip install numpy pandas --system
else:
    !uv pip install numpy pandas

Audited 2 packages in 0.83ms


In [4]:
# maybe enable logging
if LOGGING_ENABLED:
    enable_console_logging(LOGGING_LEVEL)

## LLMs

In [5]:
if IS_ON_RUNPOD:
    backbone_llm = os.getenv("OLLAMA_MODEL")
    judge_llm = "gpt-5" if os.getenv("OPENAI_API_KEY") else backbone_llm
else:
    backbone_llm = "qwen3:8b"
    judge_llm = "gpt-5" if os.getenv("OPENAI_API_KEY") else backbone_llm

In [6]:
print(f"Backbone LLM: {backbone_llm}")
print(f"Judge LLM: {judge_llm}")

Backbone LLM: qwen3:8b
Judge LLM: gpt-5


## Build Tools

In [7]:
import uuid

import numpy as np
from pydantic import BaseModel, ConfigDict, Field, computed_field

from llm_agents_from_scratch.tools import PydanticFunctionTool

### Tool: `generate_random_sample()`

In [8]:
# Global registry to store samples
SAMPLE_REGISTRY: dict[str, list[tuple[float, float]]] = {}

In [9]:
class RandomSampleParams(BaseModel):
    """Params for generate_random_sample."""

    model_config = ConfigDict(extra="forbid")
    n: int = Field(description="The number of random points to generate")


class RandomSample(BaseModel):
    """Result from generate_random_sample."""

    sample_id: str = Field(
        description="Pass this sample_id to monte_carlo_estimate",
    )

    @computed_field
    @property
    def sample_size(
        self,
    ) -> int:
        """Determine n from SAMPLE_REGISTRY."""
        return len(SAMPLE_REGISTRY[self.sample_id])


def generate_random_sample(params: RandomSampleParams) -> RandomSample:
    """Generate n random points in [-1, 1] × [-1, 1].

    Returns a sample_id. Pass this sample_id directly to monte_carlo_estimate.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 2 * orig_pts - 1

    sample_id = str(uuid.uuid4())
    SAMPLE_REGISTRY[sample_id] = [tuple(pt) for pt in transformed.tolist()]

    return RandomSample(sample_id=sample_id)

In [10]:
# test generate_random_sample() function
rs = generate_random_sample(RandomSampleParams(n=5000))
rs.model_dump()

{'sample_id': '7e0c26e7-76a6-44c9-b9f1-c5a15ed01405', 'sample_size': 5000}

In [11]:
# create tool
random_sample_tool = PydanticFunctionTool(generate_random_sample)
random_sample_tool.parameters_json_schema

{'additionalProperties': False,
 'description': 'Params for generate_random_sample.',
 'properties': {'n': {'description': 'The number of random points to generate',
   'title': 'N',
   'type': 'integer'}},
 'required': ['n'],
 'title': 'RandomSampleParams',
 'type': 'object'}

### Tool: `add_more_points()`

In [12]:
class AddPointsParams(BaseModel):
    """Params for add_more_points_to_sample."""

    model_config = ConfigDict(extra="forbid")
    sample_id: str = Field(
        description="The sample_id of the sample to augment",
    )
    n: int = Field(description="The number of random points to generate")


def add_more_points_to_sample(params: AddPointsParams) -> RandomSample:
    """Add n more random points to an existing random sample.

    Returns a sample_id and the total number of points.
    """
    orig_pts = np.random.uniform(size=(params.n, 2))
    transformed = 2 * orig_pts - 1

    # augment sample
    SAMPLE_REGISTRY[params.sample_id] += [
        tuple(pt) for pt in transformed.tolist()
    ]

    return RandomSample(sample_id=params.sample_id)

In [13]:
# test add_more_points_to_sample() function
rs = add_more_points_to_sample(AddPointsParams(n=1000, sample_id=rs.sample_id))
str(rs)

"sample_id='7e0c26e7-76a6-44c9-b9f1-c5a15ed01405' sample_size=6000"

In [14]:
# create tool
add_more_points_tool = PydanticFunctionTool(add_more_points_to_sample)
add_more_points_tool.parameters_json_schema

{'additionalProperties': False,
 'description': 'Params for add_more_points_to_sample.',
 'properties': {'sample_id': {'description': 'The sample_id of the sample to augment',
   'title': 'Sample Id',
   'type': 'string'},
  'n': {'description': 'The number of random points to generate',
   'title': 'N',
   'type': 'integer'}},
 'required': ['sample_id', 'n'],
 'title': 'AddPointsParams',
 'type': 'object'}

### Tool: `monte_carlo_estimate()`

In [15]:
class MonteCarloEstimateParams(BaseModel):
    """Params for monte_carlo_estimate."""

    model_config = ConfigDict(extra="forbid")
    sample_id: str = Field(
        description="The sample_id returned by generate_random_sample",
    )


class MonteCarloEstimateResults(BaseModel):
    """Results for monte_carlo_estimate."""

    sample_id: str
    sample_size: int
    estimate: float


def monte_carlo_estimate(
    params: MonteCarloEstimateParams,
) -> MonteCarloEstimateResults:
    """Estimate pi using Monte Carlo method.

    Args:
        params: Contains sample_id from generate_random_sample.

    Returns:
        Estimate of pi (float).
    """
    points = SAMPLE_REGISTRY[params.sample_id]
    n = len(points)
    inside = sum((x**2 + y**2) < 1 for x, y in points)
    return MonteCarloEstimateResults(
        estimate=(inside / n) * 4,
        sample_id=params.sample_id,
        sample_size=n,
    )

In [16]:
pi_estimate = monte_carlo_estimate(
    MonteCarloEstimateParams(sample_id=rs.sample_id),
)
pi_estimate

MonteCarloEstimateResults(sample_id='7e0c26e7-76a6-44c9-b9f1-c5a15ed01405', sample_size=6000, estimate=3.1286666666666667)

In [17]:
monte_carlo_estimate_tool = PydanticFunctionTool(monte_carlo_estimate)
monte_carlo_estimate_tool.parameters_json_schema

{'additionalProperties': False,
 'description': 'Params for monte_carlo_estimate.',
 'properties': {'sample_id': {'description': 'The sample_id returned by generate_random_sample',
   'title': 'Sample Id',
   'type': 'string'}},
 'required': ['sample_id'],
 'title': 'MonteCarloEstimateParams',
 'type': 'object'}

## Define our LLMAgent

In [18]:
from llm_agents_from_scratch import LLMAgent
from llm_agents_from_scratch.llms import OllamaLLM

llm = OllamaLLM(backbone_llm)
llm_agent = LLMAgent(
    llm=llm,
    tools=[
        random_sample_tool,
        add_more_points_tool,
        monte_carlo_estimate_tool,
    ],
)

## Define the Task

In [19]:
from llm_agents_from_scratch.data_structures import Task

In [30]:
instruction_template = """
You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means the estimate falls in the range [3.14150, 3.14250).
Any value from 3.14150 up to (but not including) 3.14250 is a success.

Examples:
- 3.14159 ✓ (within range)
- 3.14200 ✓ (within range)
- 3.14149 ✗ (too low)
- 3.14250 ✗ (too high)

IMPORTANT: When you achieve the target, return ONLY the sample_id from the
tool response—the exact UUID string, nothing else.

<tools>
1. generate_random_sample(n) → Creates sample, returns sample_id and sample_size
2. add_more_points_to_sample(sample_id, n) → Adds n points, returns updated
   sample_size
3. monte_carlo_estimate(sample_id) → Returns pi estimate (float)
</tools>

<algorithm>
1. Call generate_random_sample(100000) to start with 100K points
2. Call monte_carlo_estimate(sample_id) to get estimate
3. Check: is the estimate between 3.1415 and 3.1425?
   - YES → Report success and STOP
   - NO → Continue to step 4
4. Call add_more_points_to_sample, doubling the points each time:
   - First add: 100 thousand
   - Second add: 200 thousand
   - Third add: 400 thousand
   - And so on, doubling each iteration
5. After adding points, go back to step 2

Exponential growth ensures faster convergence while demonstrating adaptive
sampling.
</algorithm>

<critical_rules>
- If the task is not complete, your response MUST contain a tool call
- Do not just describe what you plan to do—actually call the tool
- Do not stop until the estimate falls within the target range
- Keep track of your iteration to calculate the correct doubling amount
- NEVER fabricate tool results or invent a sample_id—only use actual
  tool responses
</critical_rules>

<final_output>
When the estimate reaches the target precision, respond with this exact JSON
structure and nothing else:

{"sample_id": "<the-actual-sample-id-from-tool-response>"}

No explanation, no markdown formatting, no code blocks—just the raw JSON.
</final_output>

Begin by calling generate_random_sample(100000).
""".strip()

In [31]:
task = Task(
    instruction=instruction_template,
)

## Perform the Task

In [32]:
handler = llm_agent.run(task, max_steps=MAX_STEPS)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means the...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means ...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      🛠️ Executing Tool Call: generate_random_sample
INFO (llm_agents_fs.TaskHandler) :      ✅ Successful Tool Call: sample_id='acb29be5-d820-4672-be65-18419a745c80' sample_size=100000
INFO (llm_agents_fs.TaskHandler) :      ✅ Step Result: <tool_call>
{"name": "monte_carlo_estimate", "arguments": {"sample_id":"acb29be5-d820-4672-be65-18419a745c80"}}
</tool_call>
INFO (llm_agents_fs.TaskHandler) :      🧠 New Step: <tool_call>
{"name": "monte_carlo_estimate", "arguments": {"sample_id":"acb29be5-d820-4672-be65-18419a745c80"}}
</tool_call>
INFO (llm_agents_

In [36]:
# if need to cancel uncomment code below
# handler.cancel()  # noqa: ERA001

In [42]:
handler.done()

True

In [43]:
handler.exception()

llm_agents_from_scratch.errors.agent.MaxStepsReachedError('Max steps reached.')

In [44]:
print(handler.rollout)

=== Task Step Start ===

💬 assistant: My current instruction is 'You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means the estimate falls in the range [3.14150, 3.14250).
Any value from 3.14150 up to (but not including) 3.14250 is a success.

Examples:
- 3.14159 ✓ (within range)
- 3.14200 ✓ (within range)
- 3.14149 ✗ (too low)
- 3.14250 ✗ (too high)

IMPORTANT: When you achieve the target, return ONLY the sample_id from the
tool response—the exact UUID string, nothing else.

<tools>
1. generate_random_sample(n) → Creates sample, returns sample_id and sample_size
2. add_more_points_to_sample(sample_id, n) → Adds n points, returns updated
   sample_size
3. monte_carlo_estimate(sample_id) → Returns pi estimate (float)
</tools>

<algorithm>
1. Call generate_random_sample(100000) to start with 100K points
2. Call monte_carlo_estimate(sample_id) to get estimate
3. Check: is the estimate between 3.1415 and 3.1425?
   

In [45]:
result = handler.exception() or handler.result()
result

llm_agents_from_scratch.errors.agent.MaxStepsReachedError('Max steps reached.')

## Evaluation

In [46]:
def estimate_has_target_precision(estimate: MonteCarloEstimateResults) -> bool:
    """Checks if the estimate achieved the desired precision.

    Target precision is 3 decimal places (3.142), meaning the estimate
    should be between 3.1415 and 3.1425.
    """
    upper_bound = 3.1425
    lower_bound = 3.1415
    return lower_bound <= estimate.estimate < upper_bound

### Task Success

In [47]:
import json
from json import JSONDecodeError

from pydantic import ValidationError

In [48]:
try:
    sample_id = json.loads(result.content)["sample_id"]
    params = MonteCarloEstimateParams(
        sample_id=sample_id,
    )
    estimate = monte_carlo_estimate(params)
    print(estimate)
    print(
        "Estimate has target precision: ",
        estimate_has_target_precision(estimate),
    )
except (ValidationError, KeyError, JSONDecodeError):
    print("The LLM agent returned an invalid `sample_id.")
except AttributeError:
    msg = "The LLM agent exceeded max steps without reaching target precision."
    print(msg)

The LLM agent exceeded max steps without reaching target precision.


### Trajectory Analysis

In [49]:
if judge_llm.startswith("gpt-"):
    from llm_agents_from_scratch.llms.openai import OpenAILLM

    trajectory_judge = OpenAILLM(model=judge_llm)
else:
    # fallback to Ollama model
    trajectory_judge = OllamaLLM(model=judge_llm)

In [50]:
class TrajectoryJudgment(BaseModel):
    """Rubric for evaluating a Monte Carlo pi estimation agent trajectory."""

    reached_target_precision: bool = Field(
        description="True if agent achieved estimate that rounds to 3.142",
    )

    completed_without_max_steps: bool = Field(
        description=(
            "True if agent completed task without hitting max steps limit"
        ),
    )

    always_added_points_before_reestimating: bool = Field(
        description=(
            "False if agent called monte_carlo_estimate consecutively more "
            "than once before adding points"
        ),
    )

    reused_sample: bool = Field(
        description=(
            "True if agent used add_more_points_to_sample to grow the sample "
            "instead of creating new samples"
        ),
    )

    no_false_completion: bool = Field(
        description=(
            "True if agent only claimed success when the actual tool result "
            "showed 3.142. False if agent claimed convergence based on a "
            "fabricated or misread estimate."
        ),
    )

    no_missed_completion: bool = Field(
        description=(
            "True if agent stopped when estimate reached 3.142. False if "
            "agent continued adding points after already achieving target."
        ),
    )

    followed_output_format: bool = Field(
        description=(
            "True if agent's final response contained only the sample_id "
            "as instructed, with no additional text or explanation."
        ),
    )

    largest_sample_size: int | None = Field(
        description=(
            "The largest sample size achieved during the trajectory, "
            "or None if not determinable from tool outputs"
        ),
    )

    summary: str = Field(
        description="One sentence summary of trajectory quality",
    )

In [51]:
judge_prompt_template = """Evaluate this Monte Carlo pi estimation trajectory.

The agent had three tools:
- `generate_random_sample(n)` - Creates NEW sample
- `add_more_points_to_sample(sample_id, n)` - Adds points to EXISTING sample
- `monte_carlo_estimate(sample_id)` - Returns pi estimate

Correct behavior:
1. Create sample once
2. Estimate → if not between 3.1415 and 3.1425,
   add points → re-estimate → repeat
3. When target reached, respond with JSON: {{"sample_id": "<uuid>"}}

Note: If final_response is "Max steps error", the agent failed to complete
the task within the allowed number of steps.

Important: When evaluating `followed_output_format`, ONLY consider the content
in <final_response>. The <trajectory> contains intermediate reasoning steps
where explanatory text is expected and acceptable. The final response should
be valid JSON with only the sample_id field.

<final_response>
{result}
</final_response>

<trajectory>
{trajectory}
</trajectory>

Evaluate and submit your judgment.""".strip()

In [52]:
trajectory_eval = await trajectory_judge.structured_output(
    prompt=judge_prompt_template.format(
        result=str(result),
        trajectory=handler.rollout,
    ),
    mdl=TrajectoryJudgment,
)

In [53]:
print(trajectory_eval.model_dump_json(indent=4))

{
    "reached_target_precision": true,
    "completed_without_max_steps": false,
    "always_added_points_before_reestimating": false,
    "reused_sample": true,
    "no_false_completion": true,
    "no_missed_completion": false,
    "followed_output_format": false,
    "largest_sample_size": 700000,
    "summary": "Agent achieved a valid estimate but failed to stop and return the required JSON, repeatedly re-estimating and ultimately hitting the max steps."
}


## Replications

In this section, we'll repeat the task multiple times to get a more robust evaluation of our LLM agent's performance.

In [37]:
handlers = []
for _ in range(NUM_REPLICATIONS):
    h = llm_agent.run(task, max_steps=MAX_STEPS)
    handlers.append(h)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means the...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means ...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means the...[TRUNCATED]
INFO (llm_agents_fs.TaskHandler) :      ⚙️ Processing Step: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means ...[TRUNCATED]
INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: You are tasked with estimating pi using Monte Carlo methods.

TARGET: Get an estimate accurate to 3 decimal places.
Success means th

In [73]:
[h.done() for h in handlers]

[True, True, True, True, True, True, True, True, True, True]

In [74]:
[str(h.exception() or h.result()) if h.done() else "Not Done" for h in handlers]

['Max steps reached.',
 'Max steps reached.',
 'Max steps reached.',
 '0ed95778-bfc1-4d2c-ae31-f9ba972c9930',
 'Max steps reached.',
 '85abc571-afbb-4acc-bbdd-d6519d77892c',
 'Max steps reached.',
 'fd176798-f5f6-4ac4-a487-f4c654e907e3',
 '85abc571-afbb-4acc-bbdd-d6519d77892c',
 '8e0e49ed-5cc0-44db-aa1f-9cf87b785073']

#### Task Success Evaluations

In [75]:
task_success = []
for handler in handlers:
    if handler.exception():
        task_success.append(0)
        continue
    result = handler.result()
    try:
        params = MonteCarloEstimateParams(
            sample_id=result.content,
        )
        estimate = monte_carlo_estimate(params)
        task_success.append(int(estimate_has_target_precision(estimate)))
    except (ValidationError, KeyError):
        # invalid sample_id provided by LLM agent—unsuccessful task
        task_success.append(0)

In [76]:
task_success

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

#### Trajectory Evaluations

In [77]:
import asyncio

In [78]:
eval_async_tasks = []
for handler in handlers:
    async_task = trajectory_judge.structured_output(
        prompt=judge_prompt_template.format(
            result=str(handler.exception() or handler.result()),
            trajectory=handler.rollout,
        ),
        mdl=TrajectoryJudgment,
    )
    eval_async_tasks.append(async_task)

trajectory_evals = await asyncio.gather(*eval_async_tasks)
trajectory_evals

[TrajectoryJudgment(reached_target_precision=True, completed_without_max_steps=False, always_added_points_before_reestimating=False, reused_sample=False, no_false_completion=True, no_missed_completion=False, followed_output_format=False, largest_sample_size=3201000, summary='Agent reached a valid 3.142 estimate but switched to a new sample, re-estimated without adding points, failed to stop with the sample_id, and ended with a max-steps failure.'),
 TrajectoryJudgment(reached_target_precision=True, completed_without_max_steps=False, always_added_points_before_reestimating=False, reused_sample=True, no_false_completion=True, no_missed_completion=False, followed_output_format=False, largest_sample_size=800000, summary='Reached target estimate using a single growing sample but then kept re-estimating, once output a wrong sample_id, and ultimately failed to stop or follow the final-output rule, hitting max steps.'),
 TrajectoryJudgment(reached_target_precision=False, completed_without_max_

### Evaluation Summary

In [79]:
import pandas as pd

from llm_agents_from_scratch.notebook_utils import set_dataframe_display_options

# sets display options for pd.DataFrame in notebooks
set_dataframe_display_options()

In [80]:
# shape eval results into a pd.DataFrame
evals_df = pd.DataFrame(
    data=[e.model_dump() for e in trajectory_evals],
)

# add task_success column
evals_df.insert(0, "task_success", task_success)

# separate summary column
summary_df = evals_df[["summary"]].copy()
evals_df = evals_df.drop(columns=["summary"])

# compute aggregations: TOTAL and AVG rows
total_row = {}
avg_row = {}

for col, dtype in evals_df.dtypes.items():
    if dtype == "bool" or pd.api.types.is_numeric_dtype(dtype):
        total_row[col] = evals_df[col].sum()
        avg_row[col] = evals_df[col].mean()
    else:
        total_row[col] = "TOTAL"
        avg_row[col] = "AVG"

# merge evaluations and aggregations dataframes
evals_df = pd.concat(
    [
        pd.DataFrame([total_row, avg_row], index=["TOTAL", "AVG"]),
        evals_df,
    ],
)

# style
evals_df.style.apply(
    lambda r: ["border-bottom: 2px solid #444"] * len(r)
    if r.name == "AVG"
    else [""] * len(r),
    axis=1,
)

,task_success,reached_target_precision,completed_without_max_steps,always_added_points_before_reestimating,reused_sample,no_false_completion,no_missed_completion,followed_output_format,largest_sample_size
TOTAL,1.000000,6.000000,5.000000,6.000000,7.000000,7.000000,7.000000,5.000000,45101000.000000
AVG,0.100000,0.600000,0.500000,0.600000,0.700000,0.700000,0.700000,0.500000,4510100.000000
0,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3201000.000000
1,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,800000.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,12800000.000000
3,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,100000.000000
4,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,13500000.000000
5,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2700000.000000
6,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,9600000.000000
7,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,800000.000000


In [81]:
summary_df

,summary
0,"Agent reached a valid 3.142 estimate but switched to a new sample, re-estimated without adding points, failed to stop with the sample_id, and ended with a max-steps failure."
1,"Reached target estimate using a single growing sample but then kept re-estimating, once output a wrong sample_id, and ultimately failed to stop or follow the final-output rule, hitting max steps."
2,"Agent iteratively added points and estimated π but re-estimated without adding at least once, created a new sample mid-run, failed to reach the target before max steps, and final output was not just the sample_id."
3,Converged immediately with a valid in-range estimate and returned only the sample_id; no additional sampling was needed.
4,"Agent reached the target using a single growing sample but repeatedly re-estimated without adding points, failed to output only the sample_id, and ultimately hit the max-steps limit."
5,"Agent converged correctly using a single sample and proper add-then-estimate cycles, but the final response returned a different sample_id than the one used, despite correct output-only format."
6,"The agent created one sample and correctly alternated adding points and re-estimating, but never reached the target, lost track of the sample_id leading to an invalid tool call, hit max steps, and the final response did not follow the required output format."
7,"Agent created one sample and iteratively added points, but incorrectly claimed success at 3.141465 (not within target and rounds to 3.141), though the final response format was correct; largest sample size was 800000."
8,"Converged using a single growing sample and stopped appropriately, but incorrectly returned a different sample_id despite correct output-only formatting."
9,"Agent reused one sample and alternated add/estimate correctly with proper final formatting, but falsely declared success at 3.141395, which is below the target window."


In [ ]:
# write results to json
evals_df.to_json("evals_df.json")
summary_df.to_json("summary_df.json")